In [ ]:
import torch

In [21]:
batch_size = 64
seq_len = 100
emb_dim = 512

n_heads = 8
n_heads_kv = n_heads // 2
n_hid = 256
seq_len_kv = 25

### MHA

In [4]:
emb = torch.randn(batch_size, seq_len, emb_dim)

W_q = torch.randn(n_heads, 1, emb_dim, n_hid // n_heads)
W_k = torch.randn(n_heads, 1, emb_dim, n_hid // n_heads)
W_v = torch.randn(n_heads, 1, emb_dim, n_hid // n_heads)

In [5]:
Q = emb @ W_q
K = emb @ W_v
V = emb @ W_k

In [6]:
Q.shape, K.shape, V.shape

(torch.Size([8, 64, 100, 32]),
 torch.Size([8, 64, 100, 32]),
 torch.Size([8, 64, 100, 32]))

In [ ]:
att_w = Q @ K.transpose(2, 3) / (n_hid // n_heads) ** 0.5
att_w = torch.softmax(att_w, dim=-1)
att_w.shape

torch.Size([8, 64, 100, 100])

In [20]:
att_out = att_w @ V
att_out = att_out.permute(1, 2, 0, 3).reshape(batch_size, seq_len, n_hid)
att_out.shape

torch.Size([64, 100, 256])

### GQA

In [34]:
emb_enc = torch.randn(batch_size, seq_len_kv, emb_dim)
emb_dec = torch.randn(batch_size, seq_len, emb_dim)

W_q = torch.randn(n_heads, 1, emb_dim, n_hid // n_heads)
W_k = torch.randn(n_heads_kv, 1, emb_dim, n_hid // n_heads)
W_v = torch.randn(n_heads_kv, 1, emb_dim, n_hid // n_heads)

In [35]:
W_q.shape, W_k.shape, W_v.shape

(torch.Size([8, 1, 512, 32]),
 torch.Size([4, 1, 512, 32]),
 torch.Size([4, 1, 512, 32]))

In [36]:
Q = emb_dec @ W_q
K = torch.repeat_interleave(emb_enc @ W_k, repeats=n_heads//n_heads_kv, dim=0)
V = torch.repeat_interleave(emb_enc @ W_v, repeats=n_heads//n_heads_kv, dim=0)

In [37]:
Q.shape, K.shape, V.shape

(torch.Size([8, 64, 100, 32]),
 torch.Size([8, 64, 25, 32]),
 torch.Size([8, 64, 25, 32]))

In [39]:
att_w = Q @ K.transpose(2, 3) / (n_hid // n_heads)*0.5
att_w = torch.softmax(att_w, dim=-1)
att_w.shape

torch.Size([8, 64, 100, 25])

In [42]:
att_out = att_w @ V
att_out = att_out.permute(1, 2, 0, 3).reshape(batch_size, seq_len, n_hid)
att_out.shape

torch.Size([64, 100, 256])